In [9]:
#Imports
import cv2
import numpy as np
import torch
import torch.nn as nn
from torchsummary import summary
import torchvision.transforms as transforms
import copy

if torch.cuda.is_available():
	print("PyTorch is using the GPU")
	GPUCount = torch.cuda.device_count()
	print(f"Found {GPUCount} GPUs")

	for i in range(GPUCount):
		print(f"GPU {i} found: {torch.cuda.get_device_name(i)}")

	device = torch.device("cuda:0")
else:
	print("PyTorch is using the CPU")
	device = torch.device("cpu")

print(f"Selected Device: {device}")

PyTorch is using the GPU
Found 1 GPUs
GPU 0 found: NVIDIA GeForce RTX 5070 Laptop GPU
Selected Device: cuda:0


In [ ]:
#Image Preprocessing

#Kernels
VerticalKernel = cv2.getStructuringElement(cv2.MORPH_RECT , (1,21))
HorizontalKernel = cv2.getStructuringElement(cv2.MORPH_RECT , (21,1))

#Images
InputImg = cv2.imread("../TestQuestions/TestQuestion7.png")
InputImgGS = cv2.cvtColor(InputImg , cv2.COLOR_BGR2GRAY)
_,InputImgThresh = cv2.threshold(InputImgGS,127,255,cv2.THRESH_BINARY_INV)

In [11]:
#Grid Detection

Contours,Heirarchy = cv2.findContours(InputImgThresh , cv2.RETR_EXTERNAL , cv2.CHAIN_APPROX_SIMPLE)
PuzzleContours = None

if Contours:
	Contours = sorted(Contours , key=cv2.contourArea , reverse=True)
	for Contour in Contours:
		Perimeter = cv2.arcLength(Contour , True)
		Approx = cv2.approxPolyDP(Contour , Perimeter*0.02 , True)
		if (len(Approx) == 4):
			PuzzleContours = Approx
			break

#Puzzle Cropped Perfectly
if (PuzzleContours is None):
	print("The Puzzle is cropped perfectly")
	x,y,w,h = 0,0,InputImg.shape[1],InputImg.shape[0]
	DetectedBoardImage = InputImgThresh
#Contour Found
else:
	print("Cropping puzzle")
	x,y,w,h = cv2.boundingRect(PuzzleContours)
	DetectedBoardImage = InputImgThresh[y:y+h , x:x+w]
	print(f"Bounds: x={x},y={y},w={w},h={h}")

Cropping puzzle
Bounds: x=35,y=21,w=1000,h=1000


In [12]:
#Removing Grid Lines

DetectedBoardImage = cv2.resize(DetectedBoardImage , (450,450))
OutputImg = InputImg[y:y+h , x:x+w]
OutputImg = cv2.resize(OutputImg , (450,450))

#Remove Horizontal Lines
Temp = cv2.morphologyEx(DetectedBoardImage,cv2.MORPH_OPEN,HorizontalKernel,iterations=2)
contours,_ = cv2.findContours(Temp,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
for c in contours:
	cv2.drawContours(DetectedBoardImage,[c],-1,(0,0,0),5)

#Remove Vertical Lines
Temp = cv2.morphologyEx(DetectedBoardImage,cv2.MORPH_OPEN,VerticalKernel,iterations=2)
contours,_ = cv2.findContours(Temp,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
for c in contours:
	cv2.drawContours(DetectedBoardImage,[c],-1,(0,0,0),5)

# #Show Images
# cv2.imshow("Original" , InputImg)
# cv2.imshow("Grey" , InputImgGS)
# cv2.imshow("Thresh" , InputImgThresh)
# cv2.imshow("Board" , DetectedBoardImage)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [13]:
#Load model

print("Loading Model")
#Define architecture
class MNISTModel(nn.Module):
	def __init__(self):
		super(MNISTModel,self).__init__()

		#Layer1
		self.conv1 = nn.Conv2d(in_channels=1,out_channels=32,kernel_size=3,padding=1,bias=False)#Size does not change
		self.bn1 = nn.BatchNorm2d(num_features=32)
		self.relu1 = nn.ReLU()
		self.pool1 = nn.MaxPool2d(kernel_size=2)#Size halves into 14x14

		#Layer2
		self.conv2 = nn.Conv2d(in_channels=32,out_channels=32,kernel_size=3,padding=1,bias=False)
		self.bn2 = nn.BatchNorm2d(num_features=32)
		self.relu2 = nn.ReLU()

		#Layer3
		self.conv3 = nn.Conv2d(in_channels=32,out_channels=64,kernel_size=3,padding=1,bias=False)
		self.bn3 = nn.BatchNorm2d(num_features=64)
		self.relu3 = nn.ReLU()

		#Layer4
		self.conv4 = nn.Conv2d(in_channels=64,out_channels=64,kernel_size=3,padding=1,bias=False)
		self.bn4 = nn.BatchNorm2d(num_features=64)
		self.relu4 = nn.ReLU()

		#Layer5
		self.conv5 = nn.Conv2d(in_channels=64,out_channels=64,kernel_size=3,padding=1,bias=False)
		self.bn5 = nn.BatchNorm2d(num_features=64)
		self.relu5 = nn.ReLU()
		self.pool2 = nn.MaxPool2d(kernel_size=2)#Size halves into 7x7

		#FlattenLayer
		self.flatten = nn.Flatten()

		#Layer6
		self.fc1 = nn.Linear(in_features=64*7*7,out_features=64)
		self.relu6 = nn.ReLU()

		#Layer7
		self.fc2 = nn.Linear(in_features=64,out_features=32)
		self.relu7 = nn.ReLU()

		#DropoutLayer
		self.dropout = nn.Dropout(p=0.2)

		#Layer8
		self.fc3 = nn.Linear(in_features=32,out_features=10)

	def forward(self,x):

		#Pass through Layer1
		x = self.pool1(self.relu1(self.bn1(self.conv1(x))))

		#Pass through Layer2
		x = self.relu2(self.bn2(self.conv2(x)))

		#Pass through Layer3
		x = self.relu3(self.bn3(self.conv3(x)))

		#Pass through Layer4
		x = self.relu4(self.bn4(self.conv4(x)))

		#Pass through Layer5
		x = self.pool2(self.relu5(self.bn5(self.conv5(x))))

		#Pass through Layer5
		x = self.flatten(x)

		#Pass through Layer6
		x = self.relu6(self.fc1(x))

		#Pass through Layer7
		x = self.relu7(self.fc2(x))

		#Pass through DropoutLayer
		x = self.dropout(x)

		#Pass through Layer7
		x = self.fc3(x)

		#Return Prediction
		return x
	
model = MNISTModel().to(device)
model.load_state_dict(torch.load("MNISTModel.pth"))
model.eval()

print("Model Summary: ")
summary(model,input_size=(1,28,28))

Transform = transforms.Compose(
	[
		# transforms.Grayscale(num_output_channels=1),
		transforms.ToTensor(),
  		transforms.Normalize((0.5,),(0.5,))
	])

Loading Model
Model Summary: 
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             288
       BatchNorm2d-2           [-1, 32, 28, 28]              64
              ReLU-3           [-1, 32, 28, 28]               0
         MaxPool2d-4           [-1, 32, 14, 14]               0
            Conv2d-5           [-1, 32, 14, 14]           9,216
       BatchNorm2d-6           [-1, 32, 14, 14]              64
              ReLU-7           [-1, 32, 14, 14]               0
            Conv2d-8           [-1, 64, 14, 14]          18,432
       BatchNorm2d-9           [-1, 64, 14, 14]             128
             ReLU-10           [-1, 64, 14, 14]               0
           Conv2d-11           [-1, 64, 14, 14]          36,864
      BatchNorm2d-12           [-1, 64, 14, 14]             128
             ReLU-13           [-1, 64, 14, 14]               0
         

In [14]:
#Cell Segmentation and prediction

BoardHeight,BoardWidth = DetectedBoardImage.shape[:2]
CellHeight = BoardHeight//9
CellWidth = BoardWidth//9

Cells = []

for j in range(9):
	CellRow = []
	for i in range(9):
		x1 = i*CellWidth
		y1 = j*CellHeight
		x2 = (i+1)*CellWidth
		y2 = (j+1)*CellHeight

		CellImageTest = DetectedBoardImage[y1:y2 , x1:x2]
		if np.count_nonzero(CellImageTest) < 20:
			Num = 0
		else:
			CellImage = cv2.resize(CellImageTest , (28,28) , interpolation=cv2.INTER_AREA)
			CellImage = Transform(CellImage)
			CellImage = CellImage.unsqueeze(0)
			CellImage = CellImage.to(device)
			with torch.no_grad():
				output = model(CellImage)
				_,prediction = torch.max(output,dim=1)
				Num = prediction.item()

		CellRow.append(Num)
	Cells.append(CellRow)
print("Puzzle Detected: ")
print(Cells)
cv2.destroyAllWindows()

Puzzle Detected: 
[[0, 1, 5, 8, 3, 0, 0, 0, 0], [0, 0, 9, 0, 0, 5, 0, 2, 8], [4, 2, 8, 1, 0, 0, 5, 0, 0], [1, 0, 0, 2, 0, 0, 4, 0, 0], [0, 0, 0, 0, 8, 0, 0, 5, 1], [0, 0, 4, 6, 0, 3, 2, 0, 7], [5, 0, 0, 3, 0, 0, 6, 0, 0], [3, 7, 0, 4, 2, 0, 0, 1, 0], [0, 0, 0, 5, 6, 1, 7, 3, 4]]


In [15]:
#Solve Puzzle

def FindNextEmpty(Puzzle):
	for row in range(9):
		for col in range(9):
			if Puzzle[row][col] == 0:
				return row,col
	return None,None

def IsValid(Puzzle , guess , row , col):
	#Row
	if guess in Puzzle[row]:
		return False
	
	#Column
	Col = []
	for i in range(9):
		Col.append(Puzzle[i][col])
	if guess in Col:
		return False
	
	#Grid
	RowStart = (row//3)*3
	ColStart = (col//3)*3

	for r in range(RowStart , RowStart+3):
		for c in range(ColStart , ColStart+3):
			if guess == Puzzle[r][c]:
				return False

	return True

def SolvePuzzle(Puzzle):
	row,col = FindNextEmpty(Puzzle)
	
	if row is None:
		return True
	
	for guess in range(1,10):
		if (IsValid(Puzzle,guess,row,col)):
			Puzzle[row][col] = guess

			if SolvePuzzle(Puzzle):
				return True
			
		Puzzle[row][col] = 0
	return False

print("Original Board: ")
for r in range(9):
	for c in range(9):
		print(Cells[r][c] , end='   ')
	print("\n")
print("\n\n\n")
SolvedCells = copy.deepcopy(Cells)
print("Solved Board: ")
SolvePuzzle(SolvedCells)
for r in range(9):
	for c in range(9):
		print(SolvedCells[r][c] , end = '   ')
	print("\n")
print("\n\n\n")

Original Board: 
0   1   5   8   3   0   0   0   0   

0   0   9   0   0   5   0   2   8   

4   2   8   1   0   0   5   0   0   

1   0   0   2   0   0   4   0   0   

0   0   0   0   8   0   0   5   1   

0   0   4   6   0   3   2   0   7   

5   0   0   3   0   0   6   0   0   

3   7   0   4   2   0   0   1   0   

0   0   0   5   6   1   7   3   4   





Solved Board: 
7   1   5   8   3   2   9   4   6   

6   3   9   7   4   5   1   2   8   

4   2   8   1   9   6   5   7   3   

1   8   3   2   5   7   4   6   9   

2   6   7   9   8   4   3   5   1   

9   5   4   6   1   3   2   8   7   

5   4   1   3   7   8   6   9   2   

3   7   6   4   2   9   8   1   5   

8   9   2   5   6   1   7   3   4   







In [16]:
#Display Output on img

Height,Width = OutputImg.shape[:2]
CellHeight = Height//9
CellWidth = Width//9

for row in range(9):
	for col in range(9):
		if Cells[col][row] == 0:
			val = SolvedCells[col][row]
			x = row*CellWidth
			y = col*CellHeight

			Font = cv2.FONT_HERSHEY_SIMPLEX
			FontScale = 1
			Thickness = 2
			Text = str(val)
			TextSize = cv2.getTextSize(Text,Font,FontScale,Thickness)[0]
			TextX = x + (CellWidth - TextSize[0])//2
			TextY = y + (CellHeight + TextSize[1])//2

			cv2.putText(OutputImg,Text,(TextX,TextY),Font,FontScale,(255,0,0),Thickness,cv2.LINE_AA)

cv2.imshow("Result",OutputImg)
cv2.waitKey(0)
cv2.destroyAllWindows()